# Browser-use: 자율 탐색 도구 마스터하기

이번 실습에서 메인으로 다룰 무기는 **Browser-use**라는 오픈소스 프레임워크입니다. (공식 GitHub: [browser-use](https://github.com/browser-use/browser-use))

Browser-use는 에이전트가 사람처럼 웹 브라우저를 직접 제어할 수 있게 해주는 파이썬 라이브러리입니다. 단순히 데이터를 긁어오는 '크롤링'을 넘어, AI가 웹사이트의 버튼을 클릭하고, 정보를 검색하고, 복잡한 워크플로우를 스스로 수행하게 만드는 웹 에이전트(Web Agent) 개발의 핵심 도구입니다.

### 주요 기능

- 지능적 브라우징: 정해진 스크립트가 아니라, AI가 화면을 보고 다음 행동(클릭, 입력, 스크롤 등)을 스스로 판단합니다.
- Playwright 기반: 강력한 브라우저 자동화 도구인 Playwright를 사용하여 Chrome, Firefox 등 다양한 브라우저를 제어합니다.
- 간편한 통합: LangChain과 같은 프레임워크와 잘 연동되며, Google, OpenAI, Anthropic의 최신 모델을 비전(Vision) 기능을 통해 활용합니다.
- 멀티 탭 지원: 여러 탭을 동시에 관리하며 복잡한 정보를 비교하거나 수집할 수 있습니다.

### 작동 원리

- 관찰(Observe): 현재 웹 페이지의 HTML 구조와 스크린샷을 찍어 AI에게 전달합니다.
- 생각(Think): AI가 사용자의 목표(예: "가장 저렴한 항공권 예약해줘")와 현재 페이지 상태를 분석합니다.
- 행동(Act): 분석 결과를 바탕으로 클릭, 타이핑 등 구체적인 브라우저 액션을 실행합니다.


### CUA(Computer-Use Agent)의 일종

우리가 쓸 `Browser-use`가 바로 이 CUA 개념을 웹 브라우저 환경에 구현한 겁니다. *"다나와 가서 RTX 4090 최저가 모델 3개만 찾아와"* 라고 자연어로 던지면, 알아서 다나와 켜고, 검색하고, 클릭해서 결과를 가져옵니다.

.env 파일을 로드하여 환경변수에 등록합니다. 노트북을 위한 실행 경로를 세팅합니다.

In [ ]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)
#load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

In [ ]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
os.environ["DISPLAY"] = ":1"

### 첫 번째 자율 탐색: 네이버 뉴스 요약하기

자, 이제 컴퓨터에게 첫 번째 임무를 줘보겠습니다. 
아래 코드는 아무런 HTML 태그 분석 없이, 단순히 **"네이버 뉴스에 들어가서 기사 하나를 요약해 줘"**라는 자연어 명령만을 내립니다. 
`Agent`는 다음과 같은 과정을 스스로 거칩니다:

1. 브라우저를 열고 네이버 뉴스에 접속 (Action)
2. 뉴스 목록을 확인 (Observe)
3. 흥미로운 기사를 클릭 (Act)
4. 기사 본문을 읽고 요약 (Think & Done)
Jupyter Notebook 환경에서는 비동기 충돌이 일어날 수 있으므로 `nest_asyncio`를 적용했습니다. 코드를 실행하고 브라우저가 어떻게 스스로 움직이는지, 그리고 로그(Log) 창에서 AI가 어떤 '생각(Thought)'을 거쳐 행동하는지 관찰해 보세요!

In [ ]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import asyncio


async def basic_query():
    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘"

    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720} # 브라우저 크기 조절
        )

    agent = Agent(task=task, llm=llm, browser=browser)
    result = await agent.run()
    return result

if __name__ == "__main__":
    result = asyncio.run(basic_query())

In [ ]:
# history: 여러 개의 AgentHistory 객체들이 담긴 리스트
history_list = result.history 

# 총 몇 스텝(Step)을 거쳤는지 확인
print(f"총 {len(result.history)} 스텝을 진행했습니다.")


In [ ]:
history_list[0].model_dump()

In [ ]:
# 마지막 스텝 정보 가져오기
last_step = result.history[-1]

print("1) LLM이 내놓은 모델의 상태 (생각/추론 과정)")
last_model_output = last_step.model_output
if last_model_output:
    print("당시 기억 : ", last_model_output.current_state.memory)  # 에이전트의 당시 기억
    print("다음 목표 : ", last_model_output.current_state.next_goal) # 에이전트의 다음 목표

print("2) LLM이 선택한 액션(명령어)")
if last_model_output and last_model_output.action:
    # 어떤 액션을 했는지 딕셔너리 형태로 변환해서 보기
    print([action.model_dump(exclude_unset=True) for action in last_model_output.action])

print("3) 에이전트가 위치했던 URL")
print(last_step.state.url) # 에이전트가 위치했던 URL

print("4) 액션 수행 후 결과")
for res in last_step.result:
    print(res.extracted_content) # 화면에서 추출한 글자 내용 
    print(res.success) # 작업 성공 여부


최종 결과만 추출

browser-use 에이전트는 실행 결과를 깔끔하게 정리해주는 메서드를 제공합니다.

In [ ]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(result.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(result.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(result.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(result.urls())


### 비용 관리: 분석 비용 트래킹하기

In [ ]:
from utils.cost_tracker import CostTracker

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use1():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use1())

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use2():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-2.0-flash")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use2())

### 심화 기능 활용하기

##### (1) 기존 브라우저 연결

지금까지는 AI가 실행될 때마다 아무 기록이 없는 '새 브라우저(샌드박스)'를 띄웠습니다. 하지만 실무에서는 이미 로그인이 다 되어 있는 내 컴퓨터의 브라우저를 그대로 조종해야 할 때가 많습니다. 

이 코드는 **현재 여러분 컴퓨터에 설치된 실제 구글 크롬**을 찾아 연결합니다. (미리 크롬 브라우저를 모두 닫아두고 실행해 주세요.)

In [ ]:
Browser.list_chrome_profiles()

In [ ]:
def get_default_chrome_profile() -> str | None:
    profiles = Browser.list_chrome_profiles()
    
    if not profiles:
        print("❌ 설치된 크롬 프로필을 찾을 수 없습니다.")
        return None
        
    print(f"✅ 발견된 크롬 프로필: {profiles[0]['name']}")
    return profiles[0]['directory']

In [ ]:
get_default_chrome_profile()

In [ ]:
async def real_browser_query():
    # 1. 내 시스템 기반의 Browser 객체 생성
    profile_path = get_default_chrome_profile()
    
    # 2. 에이전트 실행 시 생성한 브라우저 객체를 주입
    browser = Browser.from_system_chrome(profile_directory=profile_path)
    llm = ChatGoogle(model="gemini-2.0-flash")
    
    task = "아마존(amazon.com)에 들어가서 화이트보드 마커펜을 검색해 줘."
    
    agent = Agent(
        task=task, 
        llm=llm, 
        browser=browser
    )
    
    result = await agent.run()
    
    # 세션 종료 후 브라우저 닫기
    await browser.stop()
    return result

result = asyncio.run(real_browser_query())
print(result.final_result())

##### (2) 브라우저를 끄지 않고 연속해서 작업 (keep_alive=True)



In [ ]:
profile_dir = os.path.join(os.getcwd(), "my_quotes_profile")

browser = Browser(
        headless=False,
        disable_security=True,
        window_size={'width': 1280, 'height': 720},
        user_data_dir=profile_dir,
        keep_alive=True
    )

# 2. LLM 및 에이전트 설정
llm = ChatGoogle(model="gemini-2.0-flash")
task = "http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야."

agent = Agent(
        task=task, 
        llm=llm,
        browser=browser)

history = await agent.run()
print(history.final_result())

In [ ]:
task = "http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 현재 로그인된 상태가 맞는지 결과만 짧게 대답해 줘."
    
agent2 = Agent(
        task=task, 
        llm=llm,
        browser=browser)

history = await agent2.run()
print(history.final_result())

In [ ]:
await browser.stop() # 브라우저 세션 종료

다음으로 로그인 정보(쿠키 등)를 파일로 저장하겠습니다.

In [ ]:
import os
from browser_use import Agent, Browser
from browser_use import ChatGoogle

llm = ChatGoogle(model="gemini-2.0-flash")

# keep_alive = True => 작업 끝나도 브라우저 종료 X
browser = Browser(
    headless=False,
    disable_security=True,
    window_size={'width': 1280, 'height': 720},
    keep_alive=True
)

agent = Agent(
    task="http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야.",
    llm=llm,
    browser=browser
)
history = await agent.run()
print("로그인 결과:", history.final_result())

# 현재 로그인 세션(쿠키, 로컬 스토리지 등)을 JSON 파일로 저장합니다.
await browser.export_storage_state('storage_state.json')

# ✅ 쿠키를 디스크에 저장하기 위해 명시적으로 종료
await browser.stop()
print("✅ 브라우저 종료 & 쿠키 저장 완료")


이제 다시 접속해보겠습니다.

In [ ]:
import os
from browser_use import Agent, Browser
from browser_use import ChatGoogle

profile_dir = os.path.join(os.getcwd(), "my_quotes_profile")
llm = ChatGoogle(model="gemini-2.0-flash")

# ✅ 같은 profile_dir로 새 브라우저 생성 → 저장된 쿠키 로드
browser2 = Browser(
    headless=False,
    disable_security=True,
    window_size={'width': 1280, 'height': 720},
    storage_state="storage_state.json"
)

agent2 = Agent(
    task="http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 로그인 상태인지 짧게 대답해줘.",
    llm=llm,
    browser=browser2
)
history2 = await agent2.run()
print("로그인 상태:", history2.final_result())


##### (3) 데이터 form 전송

이번에는 정보를 폼에 채워서 전송해봅시다.

In [ ]:
async def run_form_filling():
    # 1. 빈 브라우저 객체 생성

    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
    )
    
    # 2. LLM 및 에이전트 설정
    llm = ChatGoogle(model="gemini-2.0-flash")
    task = """
    https://httpbin.org/forms/post 에 접속해서 다음 정보로 폼을 채워줘:
    - 이름: 홍길동
    - 전화번호: 010-1234-5678
    - 이메일: test@example.com
    - 사이즈: Medium / 토핑: cheese / 시간: 지금
    - 의견: Browser-use 테스트 중입니다.
    
    폼을 제출한 후 서버로부터 받은 응답 내용을 요약해서 알려줘.
    """
    
    agent = Agent(
        task=task, 
        llm=llm,
        browser=browser)
    
    # 3. Agent 실행 후 결과(history) 반환받기
    history = await agent.run()

    # 4. 브라우저 닫기
    await browser.stop()
    
    return history
if __name__ == '__main__':
    history = asyncio.run(run_form_filling())

In [ ]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(history.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(history.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(history.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(history.urls())


##### (4) 페이지 내 데이터 추출

기본적인 데이터 추출을 명령해봅니다.

In [ ]:
async def run_data_extraction():
    
    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
    )
    
    
    llm = ChatGoogle(model='gemini-2.0-flash')
    
    # 5개의 명언, 저자, 태그를 특정 형식으로 추출 요청
    task = """
    https://quotes.toscrape.com/ 에서 다음 정보를 추출해줘:
    - 페이지에 있는 첫 5개의 명언
    - 각 명언의 저자와 태그들
    
    결과는 '명언 1: [내용] - 저자: [이름] - 태그: [태그들]' 형식으로 정리해줘.
    """
    
    agent = Agent(task=task, llm=llm, browser=browser)
    history = await agent.run()
    return history

if __name__ == '__main__':
    history = asyncio.run(run_data_extraction())

In [ ]:
history

In [ ]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(history.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(history.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(history.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(history.urls())


##### (5) 에이전트의 빠른 브라우징 작업

에이전트의 불필요한 생각 시간을 줄이고 브라우저 동작 속도를 극대화하는 설정입니다.

- flash_mode=True: AI의 내부 사고 과정을 생략하여 응답 속도를 높입니다.
- Browser: 페이지 로딩 대기 시간과 액션 간격을 0.1초로 단축합니다.
- extend_system_message: AI에게 "최대한 짧고 빠르게 행동하라"는 지침을 추가합니다.

In [ ]:
from browser_use import BrowserProfile

async def run_fast_agent():
    llm = ChatGoogle(model='gemini-flash-latest')
    
    # 1. 속도 최적화 브라우저 설정   
    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절
            wait_between_actions=0.1,        # 동작 간격 단축
            minimum_wait_page_load_time=0.1 # 최소 대기 시간 단축
    )


    # 2. 모델에게 내리는 '빨리빨리' 명령
    optimization_prompt = "최대한 간결하게 응답하고, 가능한 여러 동작을 한 번에 실행해서 목표에 빠르게 도달해."

    agent = Agent(
        task="네이버 뉴스 상위 3개 기사 제목만 빠르게 뽑아줘.",
        llm=llm,
        flash_mode=True,                # 사고 과정 생략
        browser=browser,
        extend_system_message=optimization_prompt
    )
    
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_fast_agent())

##### (6) 비디오 녹화

작업 과정을 녹화합니다. AI가 어떻게 움직였는지 사후에 검토하거나 디버깅할 때 매우 유용합니다.

In [ ]:
import asyncio
from pathlib import Path
from browser_use import Agent, Browser, ChatGoogle
from browser_use.browser import BrowserProfile
from dotenv import load_dotenv

load_dotenv()

async def main():
    # 1. 영상이 저장될 폴더 설정
    video_dir = Path('./recordings')
    video_dir.mkdir(exist_ok=True)
    
    # 2. 브라우저 설정 (핵심: record_video_dir)
    # 구조를 따름
    browser = Browser(
        headless=True, # VNC 화면에 브라우저 창 표시하지 않습니다.
        window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
        wait_between_actions=0.1,        # 동작 간격 단축
        minimum_wait_page_load_time=0.1, # 최소 대기 시간 단축
        record_video_dir=video_dir,  # 영상 저장 경로 지정
    )

    # 4. LLM 및 에이전트 설정
    llm = ChatGoogle(model='gemini-flash-latest')
    
    task = "https://www.naver.com 에 오늘의 날씨에 대해 찾아보고 알려줘"

    agent = Agent(
        task=task,
        llm=llm,
        browser=browser,
    )

    print(f"🚀 작업을 시작합니다. 녹화본은 '{video_dir}' 폴더에 저장됩니다.")

    try:
        # 에이전트 실행 (단계 제한을 둬서 테스트)
        return await agent.run(max_steps=10)
    finally:
        print(f"✅ 작업 완료! '{video_dir}' 폴더에서 mp4 파일을 확인하세요.")

if __name__ == '__main__':
    history = asyncio.run(main())

In [ ]:
history.final_result()